In [27]:
## EMU-SV: StateVector class testing api

import torch 
from IPython.display import Latex 

from emu_sv import StateVector, inner, DenseOperator
dtype = torch.complex128

import math

#### The basics function related to StateVector class in EMU-SV

We are going to define 2 states: 


$ |\psi\rangle = \frac{1}{\sqrt 2}(|00\rangle+|11\rangle)$ and $|\phi\rangle = \frac{1}{\sqrt 2}(|01\rangle+|11\rangle)$

In [28]:

factor = math.sqrt(2.0)

basis = ("r","g")
nqubits = 2
string_state1 = {"gg":1.0,"rr":1.0}
state1 = StateVector.from_state_amplitudes(eigenstates=basis, amplitudes=string_state1)


string_state2 = {"gr":1.0/factor,"rr":1.0/factor}
state2 = StateVector.from_state_amplitudes(eigenstates=basis, amplitudes=string_state2)

In [29]:
# representation of state 1
state1

tensor([0.7071+0.j, 0.0000+0.j, 0.0000+0.j, 0.7071+0.j], device='cuda:0',
       dtype=torch.complex128)

In [30]:
#shape of the tensor and sample
print("\nShape: ",state1.vector.shape[0])
#norm of the state
print("\nnorm:",state1.norm())
display(Latex(r"Sampling $|\psi\rangle$:"))
state1.sample() # sampling the state


Shape:  4

norm: tensor(1.0000, dtype=torch.float64)


<IPython.core.display.Latex object>

Counter({'00': 510, '11': 490})

In [31]:
display(Latex(r"Inner product $\langle\psi|\phi \rangle :$"))
print(inner(state1,state2).item())
display(Latex(r"norm of $|\psi\rangle:$ "))
print(state1.norm())
display(Latex(r"$ |\delta\rangle=|\phi\rangle+2 e^{\pi i} |\psi\rangle:$"))
result_state = state1 + 2*torch.exp(torch.tensor(3.14159j))*state2
print("\nFinal state:",result_state)
print("\nsampling the resulting state")
print(result_state.sample(num_shots=1000))


<IPython.core.display.Latex object>

(0.49999999144286444+0j)


<IPython.core.display.Latex object>

tensor(1.0000, dtype=torch.float64)


<IPython.core.display.Latex object>


Final state: tensor([ 0.7071+0.0000e+00j, -1.4142+3.5853e-06j,  0.0000+0.0000e+00j,
        -0.7071+3.5853e-06j], device='cuda:0', dtype=torch.complex128)

sampling the resulting state
Counter({'01': 681, '11': 160, '00': 159})


In [32]:
# sampling the other state
sampling = state2.sample(num_shots=1000)
print(sampling)
sampling["11"]

Counter({'01': 507, '11': 493})


493

In [33]:
# sampling the state |111>+|000>
nqubits = 3
string_state = {"rrr":1.0, "ggg":1.0}
state3 = StateVector.from_state_amplitudes(eigenstates=basis, amplitudes=string_state)
state3.sample(num_shots=1000)

Counter({'000': 503, '111': 497})

In [34]:
# sampling the state |111>+|000>
nqubits = 3
string_state = {"rrr":1.0, "ggg":1.0}
state3 = StateVector.from_state_amplitudes(eigenstates=basis, amplitudes=string_state)
state3.sample(num_shots=1000)

Counter({'111': 515, '000': 485})

In [35]:
# 10 atoms in the ground state using the make function
n=10
ground = StateVector.make(n)
ground.sample()


Counter({'0000000000': 1000})

In [36]:
## use make in order to generate a ground state |000000..0>

StateVector.make(2)

tensor([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], device='cuda:0',
       dtype=torch.complex128)

In [37]:
vector = torch.tensor([0.1,1.0,1.0,1.0])
state = StateVector(vector)
state._normalize()


Operators in Emu-SV

Creating and algebra related to operators

In [38]:
z_coeff = 1.3
ops_1 = [
    (
        1.0,
        [
            ({"gr": 1.0, "rg": 1.0}, {0, 2}),  # X
            ({"gg": z_coeff, "rr": -z_coeff}, {1}),  # z_coeff*Z
        ],
    )
]

basis = ("r","g")
N = 3
operator_1 = DenseOperator.from_operator_repr(
    eigenstates=basis,
    n_qudits=N,
    operations=ops_1,
)

operator_1



tensor([[ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  1.3000+0.j,
          0.0000+0.j,  0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  1.3000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j, -1.3000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
         -1.3000+0.j,  0.0000+0.j],
        [ 0.0000+0.j,  1.3000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j],
        [ 1.3000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j, -1.3000+0.j,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j, -1.3000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j]], device='cuda:0'

In [39]:
X = torch.tensor([[0, 1], [1, 0]], dtype=dtype)
Z = torch.tensor([[1, 0], [0, -1]], dtype=dtype)

expected = torch.kron(torch.kron(X, z_coeff * Z), X)
assert torch.allclose(operator_1.matrix.cpu(), expected)

In [40]:
ops_2 = [
        (
            2.0,
            [
                ({"g"+"r":1.0, "r"+"g":1.0}, [0, 2]),
                ({"g"+"r":-1.0j, "r"+"g":1.0j}, [1]),
            ],
        )
    ]

basis = ("r","g")
N = 3
operator_2 = DenseOperator.from_operator_repr(
    eigenstates=basis,
    n_qudits=N,
    operations=ops_2,
)


In [41]:
#summing 2 operators
operator_1 + operator_2

tensor([[ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
          1.3000+0.j,  0.0000+0.j,  0.0000+2.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  1.3000+0.j,
          0.0000+0.j,  0.0000+2.j,  0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,
          0.0000-2.j,  0.0000+0.j, -1.3000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000+0.j,  0.0000-2.j,
          0.0000+0.j, -1.3000+0.j,  0.0000+0.j],
        [ 0.0000+0.j,  1.3000+0.j,  0.0000+0.j,  0.0000+2.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j,  0.0000+0.j],
        [ 1.3000+0.j,  0.0000+0.j,  0.0000+2.j,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j,  0.0000+0.j],
        [ 0.0000+0.j,  0.0000-2.j,  0.0000+0.j, -1.3000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j,  0.0000+0.j],
        [ 0.0000-2.j,  0.0000+0.j, -1.3000+0.j,  0.0000+0.j,  0.0000+0.j,
          0.0000+0.j,  0.0000+0.j,  0.0000+0.j]], device='cuda:0'

In [42]:
#scalar multiplication
5.0*operator_2

tensor([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+10.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+10.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-10.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-10.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+10.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+10.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.-10.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.-10.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
       device='cuda:0', dtype=torch.complex128)

In [43]:
#operator applied to a StateVector
state = StateVector.make(3) #|000>
operator_1.apply_to(state)

tensor([0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 0.0000+0.j, 1.3000+0.j, 0.0000+0.j,
        0.0000+0.j], device='cuda:0', dtype=torch.complex128)

In [44]:
# expectation value
expectation_000 = operator_1.expect(state)
expectation_000

tensor(0.+0.j, dtype=torch.complex128)

In [45]:
# aplication of an operator to a StateVector

basis = ("r", "g")
state = {"rrr": 1.0, "ggg": 1.0}
nqubits = 3

state_from_ampl = StateVector.from_state_amplitudes(
    eigenstates=basis,
    amplitudes=state,
)

operator_1.apply_to(state_from_ampl)

tensor([ 0.0000+0.j,  0.0000+0.j, -0.9192+0.j,  0.0000+0.j,  0.0000+0.j,  0.9192+0.j,
         0.0000+0.j,  0.0000+0.j], device='cuda:0', dtype=torch.complex128)

In [46]:
#expectation value
print(operator_1.expect(state_from_ampl))

print(operator_2.expect(state_from_ampl))

tensor(0.+0.j, dtype=torch.complex128)
tensor(0.+0.j, dtype=torch.complex128)


In [47]:
## operator - operator multipliation

operator_1 @ operator_2

tensor([[0.+0.0000j, 0.+0.0000j, 0.+2.6000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j,
         0.+0.0000j],
        [0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+2.6000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j,
         0.+0.0000j],
        [0.+2.6000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j,
         0.+0.0000j],
        [0.+0.0000j, 0.+2.6000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j,
         0.+0.0000j],
        [0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+2.6000j,
         0.+0.0000j],
        [0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j,
         0.+2.6000j],
        [0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+2.6000j, 0.+0.0000j, 0.+0.0000j,
         0.+0.0000j],
        [0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+0.0000j, 0.+2.6000j, 0.+0.0000j,
         0.+0.0000j]], device='cuda:0', dtype=torch.complex128)